In [18]:
import yfinance as yf
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import datetime
from datetime import timedelta

In [19]:
#Declaramos fechas
fecha_hoy = datetime.date.today()
fecha_hoy_ly = fecha_hoy - datetime.timedelta(days=365)
#Pasamos a string
fecha_hoy_str=fecha_hoy.strftime("%Y-%m-%d")
fecha_hoy_ly_str=fecha_hoy_ly.strftime("%Y-%m-%d")

#Datos de autenticación de mailing
smtp_server = "smtp.gmail.com"
smtp_port = 587
remitente = "tkamada07@gmail.com"
password = "awxt imrp xuvj aosh"  
destinatario = "takeshikamada@hotmail.com"
mensaje = MIMEMultipart()
mensaje["From"] = "BOT Python - Take"
mensaje["To"] = destinatario
mensaje["Subject"] = "Correo de prueba en Python"

In [20]:
# Descargar datos de Apple
data = yf.download("MSFT", start=fecha_hoy_ly_str, end=fecha_hoy_str).reset_index()

C:\Users\tk\AppData\Local\Temp\ipykernel_32864\1570433468.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("MSFT", start=fecha_hoy_ly_str, end=fecha_hoy_str).reset_index()
[*********************100%***********************]  1 of 1 completed

In [21]:
data['Ticker']=data.columns.get_level_values(1)[1]

In [22]:
# después de tu download + reset_index
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

In [23]:
df_precios=data.copy()

In [24]:
# Aseguramos que la columna Date sea de tipo fecha
df_precios["Date"] = pd.to_datetime(df_precios["Date"])
df_precios = df_precios.sort_values("Date")  # ordenar por fecha

In [25]:
# Calcular medias móviles
df_precios["SMA50"] = df_precios["Close"].rolling(50).mean()
df_precios["SMA5"] = df_precios["Close"].rolling(5).mean()

# Medias Móviles Simples

Golden Cross y Death Cross

In [26]:
# Regla simple: SMA50 vs SMA200
df_precios["SMA Cross"] = df_precios.apply(
    lambda row: "Alcista" if row["SMA5"] > row["SMA50"] else "Bajista",
    axis=1
)

In [27]:
df_precios.tail(2)

Price,Date,Close,High,Low,Open,Volume,Ticker,SMA50,SMA5,SMA Cross
247,2025-09-11,501.010010,503.170013,497.880005,502.250000,18881600,MSFT,509.427056,498.598004,Bajista
248,2025-09-12,509.899994,512.549988,503.850006,506.649994,23612600,MSFT,509.819375,501.578003,Bajista


In [28]:
if (df_precios["SMA Cross"].iloc[-2]=="Alcista") & (df_precios["SMA Cross"].iloc[-1]=="Bajista"):
    cuerpo="Inicio de Tendencia Bajista: VENDER"
elif (df_precios["SMA Cross"].iloc[-2]=="Bajista") & (df_precios["SMA Cross"].iloc[-1]=="Alcista"):
    cuerpo="Inicio de Tendencia Alcista: COMPRAR"
elif (df_precios["SMA Cross"].iloc[-2]=="Bajista") & (df_precios["SMA Cross"].iloc[-1]=="Bajista"):
    cuerpo="Se mantiene bajista"
elif (df_precios["SMA Cross"].iloc[-2]=="Bajista") & (df_precios["SMA Cross"].iloc[-1]=="Bajista"):
    cuerpo="Se mantiene alcista"
print(cuerpo)

Se mantiene bajista


In [29]:
# Cuerpo del correo
mensaje.attach(MIMEText(cuerpo, "plain"))

In [30]:
try:
    # Conectar al servidor SMTP
    servidor = smtplib.SMTP(smtp_server, smtp_port)
    servidor.starttls()  # Seguridad TLS
    servidor.login(remitente, password)
    servidor.send_message(mensaje)
    servidor.quit()
    print(f"Correo enviado exitosamente {destinatario}  ✅")
except Exception as e:
    print(f"Error al enviar el correo: {e}")

Correo enviado exitosamente takeshikamada@hotmail.com  ✅
